In [2]:
from enum import Enum

import dspy
from pydantic import BaseModel, EmailStr
from typing import Literal

# -----------------------------
# Enums for Argument Types
# -----------------------------

class QBFeatureEnum(str, Enum):
    accounts_payable = 'accounts payable'
    add_trips_manually = 'add trips manually'
    bank_statements = 'bank statements'
    budget = 'budget'
    capital = 'capital'
    categorization = 'categorization'
    certification = 'certification'
    change_business_name = 'change business name'
    connect_to_bank = 'connect to bank'
    deposits = 'deposits'
    depreciation = 'depreciation'
    import_journal_entries = 'import journal entries'
    inventory = 'inventory'
    melio = 'melio'
    overtime = 'overtime'
    payroll = 'payroll'
    purchase_order = 'purchase order'
    purchase_orders = 'purchase orders'
    reclassify = 'reclassify'
    recover_deleted_account = 'recover deleted account'
    reset_account = 'reset account'
    record_an_expense = 'record an expense'
    reconciliation = 'reconciliation'
    shortcuts = 'shortcuts'
    timesheets = 'timesheets'
    timesheets_payroll = 'timesheets/payroll'
    vendors = 'vendors'
    write_off_bad_debt = 'write off bad debt'

class ConversationTopicEnum(str, Enum):
    account = 'account'
    Accounts_Payable = 'Accounts Payable'
    Accounts_Receivable = 'Accounts Receivable'
    Accounting_Software = 'Accounting Software'
    Bank_Reconciliation = 'Bank Reconciliation'
    Billing = 'Billing'
    Bookkeeping = 'Bookkeeping'
    Budget = 'Budget'
    Budget_Tracking_and_Forecasting = 'Budget Tracking and Forecasting'
    Cash_Flow_Management = 'Cash Flow Management'
    Financial_Analysis = 'Financial Analysis'
    Financial_Planning = 'Financial Planning'
    Financial_Reporting = 'Financial Reporting'
    Fixed_Assets = 'Fixed Assets'
    insurance = 'insurance'
    Inventory_Management = 'Inventory Management'
    Invoicing = 'Invoicing'
    issue = 'issue'
    order = 'order'
    password = 'password'
    Payroll = 'Payroll'
    product = 'product'
    Purchase_Orders = 'Purchase Orders'
    questions = 'questions'
    Reconciliations = 'Reconciliations'
    returns = 'returns'
    technical = 'technical'
    shipping = 'shipping'
    service_plan = 'service_plan'
    tax = 'tax'
    Tax_Filing = 'Tax Filing'
    Vendor_Management = 'Vendor Management'




# -----------------------------
# Enums for Argument Types
# -----------------------------

class ConversationChannelEnum(str, Enum):
    speak = "speak"
    talk = "talk"
    connect = "connect"
    video = "video"
    chat = "chat"
    phone = "phone"


# -----------------------------
# Example Enum Models
# -----------------------------

class InvoiceDetailEnum(str, Enum):
    Construction_Project = 'Construction Project'
    Tutoring_Services = 'Tutoring Services'
    Website_Design = 'Website Design'
    Car_Repair = 'Car Repair'
    Catering_Services = 'Catering Services'
    Event_Management = 'Event Management'
    Graphic_Design = 'Graphic Design'
    Photography_Service = 'Photography Service'
    Marketing_Campaign = 'Marketing Campaign'
    Business_Consultation = 'Business Consultation'
    Furniture_Supplies = 'Furniture Supplies'
    Cleaning_Service = 'Cleaning Service'
    Painting_Service = 'Painting Service'
    IT_Consultancy = 'IT Consultancy'
    Accounting_Services = 'Accounting Services'
    Renovation_Work = 'Renovation Work'
    Gardening_Service = 'Gardening Service'
    Legal_Consultation = 'Legal Consultation'
    Transportation_Service = 'Transportation Service'
    Personal_Training_Services = 'Personal Training Services'

# -----------------------------
# Pydantic Models for Argument Types
# -----------------------------

class DatePeriod(BaseModel):
    start_date: str
    end_date: str

class Date(BaseModel):
    date: str

class Amount(BaseModel):
    amount: float

class GivenName(BaseModel):
    given_name: str

class FamilyName(BaseModel):
    family_name: str

class Email(BaseModel):
    email: EmailStr

class Phone(BaseModel):
    phone: str

# -----------------------------
# Combined Argument Models
# -----------------------------

class ChargeLookup(BaseModel):
    date_of_charge: Date
    amount_of_charge: Amount

class HelpRequest(BaseModel):
    help_topic: QBFeatureEnum

class ContactUsRequest(BaseModel):
    contact_topic: ConversationTopicEnum
    contact_channel: ConversationChannelEnum

class CreateInvoiceRequest(BaseModel):
    invoice_amount: Amount
    invoice_detail: InvoiceDetailEnum

class UpdateCustomerProfile(BaseModel):
    customer_given_name: GivenName
    customer_family_name: FamilyName
    customer_email: Email
    customer_phone: Phone

# -----------------------------
# Example Usage
# -----------------------------

# Example 1: DatePeriod
date_period_example = DatePeriod(
    start_date="11/01/2023",
    end_date="11/30/2023"
)

# Example 2: Amount
amount_example = Amount(
    amount=4500.00
)

# Example 3: Email
email_example = Email(
    email="j.fischer@abc.com"
)

# Example 4: Phone
phone_example = Phone(
    phone="987-654-3210"
)

# Example 5: ContactUsRequest
contact_us_request_example = ContactUsRequest(
    contact_topic=ConversationTopicEnum.Billing,
    contact_channel="chat"
)

# -----------------------------
# Printing Examples
# -----------------------------

print(date_period_example.model_dump_json())
print(amount_example.model_dump_json())
print(email_example.model_dump_json())
print(phone_example.model_dump_json())
print(contact_us_request_example.model_dump_json())


In [16]:
from dslmodel import DSLModel, init_instant, init_text
from enum import Enum
from typing import List, Annotated
from pydantic import BaseModel, Field
from openai import OpenAI
import os

MODEL = "llama3-groq-70b-8192-tool-use-preview"

client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

class Attribute(BaseModel):
    name: str
    value: str

class UI(BaseModel):
    tag: Annotated[str, Field(name="tag", description="The HTML tag for the UI element")]
    label: str
    children: Annotated[List["UI"], Field(name="children", description="The children of the UI element", default=[])]
    attributes: Annotated[List[Attribute], Field(name="attributes", description="The attributes of the UI element")]

UI.model_rebuild() # This is required to enable recursive types

init_text()

class Response(DSLModel):
    ui: UI

print(Response.from_prompt("Make a User Profile Form with 3 fields: Name, Email, Phone"))

In [17]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(DSLModel):
    steps: list[Step]
    final_answer: str

math_reasoning = MathReasoning.from_prompt("how can I solve 8x + 7 = -23")
math_reasoning

In [18]:
from pydantic import BaseModel, Field
from typing import List
from dslmodel import DSLModel

class Step(BaseModel):
    explanation: str = Field(..., description="Explanation for the step.")
    output: str = Field(..., description="Generated output for the step.")

class ASPToPDDL(DSLModel):
    str_objects: str = Field(default="", description="PDDL object definitions.")
    str_init: str = Field(default="", description="PDDL initial state.")
    str_goal: str = Field(default="", description="PDDL goal state.")
    steps: List[Step] = Field(default_factory=list, description="Steps involved in the transformation.")
    final_output: str = Field(default="", description="Final PDDL representation.")

# Example Natural Language Input
prompt = "Translate 'generate a profit and loss report for Q1 2023' into PDDL."

# Generate the model instance from the prompt
asp_to_pddl = ASPToPDDL.from_prompt(prompt)

# Output the model instance
print(asp_to_pddl)


In [19]:
# Argument Type Model
class ArgumentType(BaseModel):
    name: str = Field(
        ...,
        description="The name of the argument, representing a piece of required data for achieving the goal. Examples: 'report_period', 'date_of_charge'."
    )
    description: str = Field(
        ...,
        description="A detailed explanation of the argument's role in the goal. Example: 'Time period defined by start and end dates, accounting for leap years.'"
    )
    type: str = Field(
        ...,
        description="The data type or category of the argument. Examples: 'arg_type_date_period', 'arg_type_amount'."
    )
    
class Example(BaseModel):
    query: str = Field(
        ...,
        description="A user query demonstrating an example input for this goal. Example: 'What was the total expense for the first quarter of 2023?'."
    )
    representation: str = Field(
        ...,
        description="The intermediate representation of the query in a logical form. Example: '_goal(x, goal_2, expense_report). _report_period(x, (\"01/01/2023\", \"03/31/2023\")).'."
    )


# Goal Model
class Goal(BaseModel):
    goal_type: str = Field(
        ...,
        description="The unique identifier for the goal. Example: 'goal_2'."
    )
    description: str = Field(
        ...,
        description="A detailed explanation of the goal's purpose. Example: 'Request for generating a report on expenses and spending insights.'."
    )
    required_information: List[ArgumentType] = Field(
        default_factory=list,
        description=(
            "A list of required arguments for fulfilling this goal, "
            "each described in terms of name, type, and purpose. Example: "
            "[ArgumentType(name='report_period', description='Time period defined by start and end dates.', type='arg_type_date_period')]."
        )
    )
    examples: List[Example] = Field(
        default_factory=list,
        description=(
            "A list of example queries and their corresponding logical representations. Example: "
            "[Example(query='What was the total expense for the first quarter of 2023?', "
            "representation='_goal(x, goal_2, expense_report). _report_period(x, (\"01/01/2023\", \"03/31/2023\")).')]."
        )
    )


class DomainGoals(DSLModel):
    goals: List[Goal] = Field(
        ...,
        description=(
            "A collection of all domain goals, each describing a specific intent and its requirements. "
            "Example: goals=[Goal(goal_type='goal_2', description='Request for generating a report on expenses.', ...)]."
        )
    )

    
    
goals = [
    "Request for generating a report on one of [profit_and_loss, income, business_insights, figures, operating_income, report, revenue, earnings].",
    "Request for generating a report on one of [expense, spend, bills, operating_expense, spend_figures, spending_insight, spend_report, expense_report, business_insights].",
    "Request for generating a report on one of [earnings, pending_payments, due_accounts, invoices, invoice_report, sales, accrued_expense, financial_forecast, revenue, report].",
    "Request for one of [charge_lookup].",
    "Request for instructions on accomplishing a task in QuickBooks.",
    "Request for a conversation with a person on the best matching conversation topic using the best matching conversation medium.",
    "Request for advice about one of [business analysis, business comparison, business recommendation, personal finance, business expense, profit making].",
    "Request for creating a new invoice for a given amount and invoice detail.",
    "Request for updating a customer profile."
]


print(DomainGoals.from_prompt("Define the domain goals for a financial management system."))


In [21]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional


# --- Thought Model ---
class Thought(BaseModel):
    """
    Represents a single unit of information generated by an LLM.
    """
    id: str = Field(
        ...,
        description="Unique identifier for the thought. Example: 'T1'."
    )
    content: str = Field(
        ...,
        description="The content or information contained in this thought. Example: 'Sort the array in ascending order.'."
    )
    metadata: Optional[Dict[str, str]] = Field(
        default_factory=dict,
        description="Optional metadata about the thought, such as its type or quality score. Example: {'type': 'intermediate', 'score': '0.85'}."
    )


# --- Edge Model ---
class Edge(BaseModel):
    """
    Represents a dependency or relationship between two thoughts.
    """
    source: str = Field(
        ...,
        description="The ID of the source thought. Example: 'T1'."
    )
    target: str = Field(
        ...,
        description="The ID of the target thought. Example: 'T2'."
    )
    relation: str = Field(
        ...,
        description="The nature of the relationship or dependency. Example: 'depends_on', 'refines', 'contradicts'."
    )


# --- Transformation Model ---
class Transformation(BaseModel):
    """
    Represents a transformation applied to one or more thoughts to generate new ones.
    """
    id: str = Field(
        ...,
        description="Unique identifier for the transformation. Example: 'Tr1'."
    )
    inputs: List[str] = Field(
        ...,
        description="List of thought IDs used as inputs for the transformation. Example: ['T1', 'T2']."
    )
    operation: str = Field(
        ...,
        description="Description of the transformation operation. Example: 'Combine and summarize.'."
    )
    outputs: List[str] = Field(
        ...,
        description="List of thought IDs generated as outputs of the transformation. Example: ['T3']."
    )


# --- Graph Model ---
class GraphOfThoughts(DSLModel):
    """
    Represents the overall structure of a GoT graph, including thoughts, edges, and transformations.
    """
    thoughts: List[Thought] = Field(
        default_factory=list,
        description="List of all thoughts (vertices) in the graph."
    )
    edges: List[Edge] = Field(
        default_factory=list,
        description="List of all dependencies (edges) between thoughts in the graph."
    )
    transformations: List[Transformation] = Field(
        default_factory=list,
        description="List of all transformations applied to thoughts in the graph."
    )
    metadata: Optional[Dict[str, str]] = Field(
        default_factory=dict,
        description="Optional metadata about the graph, such as its purpose or evaluation metrics."
    )


GraphOfThoughts.from_prompt("What is saticficing for Robotic Process Automation? Think about a pydantic model to represent it")

In [25]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional


# --- Query Node ---
class QueryNode(BaseModel):
    """
    Represents a natural language query from the user.
    """
    id: str = Field(..., description="Unique identifier for the query node. Example: 'q1_2023_report'.")
    query_text: str = Field(..., description="The natural language query. Example: 'Quarterly report for Q1 2023'.")
    user_intent: str = Field(
        ..., 
        description="The intent behind the query. Example: 'generate_quarterly_report'."
    )


# --- PDDL Action Node ---
class PDDLActionNode(BaseModel):
    """
    Represents a PDDL action in the reasoning process.
    """
    id: str = Field(..., description="Unique identifier for the PDDL action node. Example: 'q1_2023_report_pddl'.")
    action_name: str = Field(..., description="The name of the PDDL action. Example: 'generate_quarterly_report'.")
    parameters: List[str] = Field(
        default_factory=list, 
        description="List of parameters for the PDDL action. Example: ['start_date', 'end_date']."
    )
    preconditions: str = Field(
        ..., 
        description="The preconditions required for the action. Example: '(and (has_date start_date))'."
    )
    effects: str = Field(
        ..., 
        description="The effects of the action when executed. Example: '(report_generated quarterly_report)'."
    )


# --- API Call Node ---
class APICallNode(BaseModel):
    """
    Represents an API call to fulfill the user query.
    """
    id: str = Field(..., description="Unique identifier for the API call node. Example: 'q1_2023_api_call'.")
    api_name: str = Field(..., description="The name of the API being invoked. Example: 'generate_report_api'.")
    input_parameters: Dict[str, str] = Field(
        default_factory=dict, 
        description="Key-value pairs of input parameters. Example: {'start_date': '01/01/2023', 'end_date': '03/31/2023'}."
    )
    output: Optional[str] = Field(
        None, 
        description="The result returned by the API. Example: 'Quarterly report generated successfully.'."
    )


# --- Transformation Process ---
class TransformationProcess(BaseModel):
    """
    Represents a reasoning or transformation process between nodes in the graph.
    """
    id: str = Field(..., description="Unique identifier for the transformation process. Example: 'q1_2023_to_pddl'.")
    input_nodes: List[str] = Field(
        ..., 
        description="IDs of the input nodes for the transformation. Example: ['q1_2023_report']."
    )
    transformation_type: str = Field(
        ..., 
        description="The type of transformation, e.g., 'query_to_pddl', 'pddl_to_api'."
    )
    output_nodes: List[str] = Field(
        ..., 
        description="IDs of the output nodes from the transformation. Example: ['q1_2023_report_pddl']."
    )


# --- Graph of Thoughts ---
class GraphOfThoughts(DSLModel):
    """
    Represents the complete Graph of Thoughts for reasoning and execution.
    """
    query_nodes: List[QueryNode] = Field(default_factory=list, description="List of query nodes in the graph.")
    pddl_action_nodes: List[PDDLActionNode] = Field(
        default_factory=list, description="List of PDDL action nodes in the graph."
    )
    api_call_nodes: List[APICallNode] = Field(default_factory=list, description="List of API call nodes in the graph.")
    transformations: List[TransformationProcess] = Field(
        default_factory=list, description="List of transformations between nodes in the graph."
    )

GraphOfThoughts.from_prompt("generate a profit and loss report for Q1 2023")

In [27]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional


# --- Query Node ---
class QueryNode(BaseModel):
    """
    Represents a natural language query from the user.
    """
    id: str = Field(..., description="Unique identifier for the query node. Example: 'q1_2023_query'.")
    query_text: str = Field(..., description="The natural language query. Example: 'Generate a profit and loss report for Q1 2023'.")
    user_intent: str = Field(..., description="The intent behind the query. Example: 'generate_profit_loss_report'.")
    metadata: Optional[Dict[str, str]] = Field(default_factory=dict, description="Optional metadata for traceability. Example: {'confidence': '0.95', 'timestamp': '2024-11-22T12:00:00'}.")


# --- PDDL Action Node ---
class PDDLActionNode(BaseModel):
    """
    Represents a PDDL action in the reasoning process.
    """
    id: str = Field(..., description="Unique identifier for the PDDL action node. Example: 'q1_2023_pddl'.")
    action_name: str = Field(..., description="The name of the PDDL action. Example: 'generate_profit_loss_report'.")
    parameters: List[str] = Field(default_factory=list, description="List of parameters for the PDDL action. Example: ['start_date', 'end_date'].")
    preconditions: str = Field(..., description="The preconditions required for the action. Example: '(and (has_date start_date) (has_date end_date))'.")
    effects: str = Field(..., description="The effects of the action when executed. Example: '(report_generated profit_loss_report)'.")
    metadata: Optional[Dict[str, str]] = Field(default_factory=dict, description="Optional metadata for reasoning context. Example: {'source': 'ASP_reasoning', 'priority': 'high'}.")


# --- API Call Node ---
class APICallNode(BaseModel):
    """
    Represents an API call to fulfill the user query.
    """
    id: str = Field(..., description="Unique identifier for the API call node. Example: 'q1_2023_api_call'.")
    api_name: str = Field(..., description="The name of the API being invoked. Example: 'generate_profit_loss_report_api'.")
    input_parameters: Dict[str, str] = Field(default_factory=dict, description="Key-value pairs of input parameters for the API. Example: {'start_date': '01/01/2023', 'end_date': '03/31/2023'}.")
    output: Optional[str] = Field(None, description="The result returned by the API. Example: 'Profit and loss report generated successfully.'.")
    metadata: Optional[Dict[str, str]] = Field(default_factory=dict, description="Optional metadata for execution traceability. Example: {'response_time': '200ms', 'status': 'success'}.")


# --- Transformation Process ---
class TransformationProcess(BaseModel):
    """
    Represents a reasoning or transformation process between nodes in the graph.
    """
    id: str = Field(..., description="Unique identifier for the transformation process. Example: 'q1_2023_to_pddl'.")
    input_nodes: List[str] = Field(..., description="IDs of the input nodes for the transformation. Example: ['q1_2023_query'].")
    transformation_type: str = Field(..., description="The type of transformation. Example: 'query_to_pddl', 'pddl_to_api'.")
    reasoning_details: Optional[str] = Field(None, description="Additional details about the reasoning process. Example: 'LLM inferred the mapping based on prior knowledge'.")
    output_nodes: List[str] = Field(..., description="IDs of the output nodes from the transformation. Example: ['q1_2023_pddl'].")


# --- Graph of Thoughts ---
class GraphOfThoughts(DSLModel):
    """
    Represents the complete Graph of Thoughts for reasoning and execution.
    """
    query_nodes: List[QueryNode] = Field(default_factory=list, description="List of query nodes in the graph.")
    pddl_action_nodes: List[PDDLActionNode] = Field(default_factory=list, description="List of PDDL action nodes in the graph.")
    api_call_nodes: List[APICallNode] = Field(default_factory=list, description="List of API call nodes in the graph.")
    transformations: List[TransformationProcess] = Field(default_factory=list, description="List of transformations between nodes in the graph.")
    metadata: Optional[Dict[str, str]] = Field(default_factory=dict, description="Additional metadata about the graph. Example: {'task': 'profit_loss_report', 'status': 'complete'}.")


GraphOfThoughts.from_prompt("generate a profit and loss report for Q1 2023")


In [35]:
import dspy
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import starmap

def analyze_topic_concurrently(topic: str):
    # Define the categories and their corresponding output fields.
    analyses = dict(
        tech="overview, state_of_the_art, future_directions",
        business="market_analysis, competition, strategy",
        marketing="target_audience, channels, campaigns",
        finance="budgeting, forecasting, investments",
        competitive_intelligence="competitors, market_share, trends"
    )

    # Define a worker function for processing a single category.
    def analyze_category(category, outputs):
        chain = dspy.ChainOfThought(f"topic -> {outputs}")
        return category, chain(topic=topic)

    # Use ThreadPoolExecutor for concurrency.
    with ThreadPoolExecutor() as executor:
        # Use itertools.starmap to simplify the submission of tasks.
        future_to_category = {
            executor.submit(analyze_category, category, outputs): category
            for category, outputs in analyses.items()
        }

        # Process the results as they complete.
        results = {
            future_to_category[future]: future.result() if not future.exception() else f"Error: {future.exception()}"
            for future in as_completed(future_to_category)
        }

    return results

# Example usage:
results = analyze_topic_concurrently("Microsoft Security Strategy")

# Pretty-print the results.
for category, analysis in results.items():
    print(f"Category: {category}")
    print(f"Analysis: {analysis}")
    print("-" * 50)
